In [ ]:
from kvadrati import Kvadrat
from math import comb

def packing(kvadrati): #kvadrati je seznam cifer 
    p = MixedIntegerLinearProgram(maximization=False)
    x = p.new_variable()
    y = p.new_variable()
    zg_meja = sum(kvadrati) + 1
    a = p.new_variable(binary = True) #kvadrat i ni levo od j
    b = p.new_variable(binary = True) #kvadrat j ni desno od j
    p.set_objective(p['z'])
    p.add_constraint((p['z']) >= (max(kvadrati))+kvadrati[len(kvadrati)-2])
    for i, d in enumerate(kvadrati): # i je indeks, d je dolžina stranice
        p.add_constraint(x[i] >= 0)  # želimo nenegativne koordinate
        p.add_constraint(y[i] >= 0)
        p.add_constraint(x[i] + d <= p['z']) # omejimo p['z'] z največjo pokrito koordinato
        p.add_constraint(y[i] + d <= p['z'])
        for j , g in enumerate(kvadrati):
            p.add_constraint(x[i] + d <= x[j] + zg_meja * a[i,j])
            p.add_constraint(x[j] + g <= x[i] + zg_meja * a[j,i])
            p.add_constraint(y[i] + d <= y[j] + zg_meja * b[i,j])
            p.add_constraint(y[j] + g <= y[i] + zg_meja * b[j,i])           
    for i, j in Combinations(range(len(kvadrati)), 2):
        p.add_constraint(a[i, j] + a[j, i] + b[i, j] + b[j, i] <= 3)
    z = p.solve()
    xx = p.get_values(x)
    yy = p.get_values(y)
    return (z, [(xx[i], yy[i], d) for i, d in enumerate(kvadrati)])

def narisi_rezultat_P(rezultat):
    k = Kvadrat(0,0,rezultat[0])
    rezultat1 = list(rezultat[1])
    K = k.narisi("white")
    pomoc = []
    for i in rezultat1:
        j = Kvadrat(i[0],i[1],i[2])
        pomoc.append(j)
    RISI = [K]
    for l in pomoc:
        mjau = l.narisi()
        RISI.append(mjau)
    show(sum(RISI[i] for i in range(len(RISI))))

def packing_ktimes(kvadrati,k): #kvadrati je seznam cifer, tale dela če so cifre urejene in vsako točko lahko pokrijemo k-krat
    p = MixedIntegerLinearProgram(maximization=False,solver='GLPK')
    x = p.new_variable()
    y = p.new_variable()
    zg_meja = sum(kvadrati) + 1
    a = p.new_variable(binary = True) #kvadrat i ni levo od j
    b = p.new_variable(binary = True) #kvadrat j ni desno od j
    L = p.new_variable(binary = True)
    p.set_objective(p['z'])
    p.add_constraint((p['z']) >= (max(kvadrati)))
    for i, d in enumerate(kvadrati): # i je indeks, d je dolžina stranice
        p.add_constraint(x[i] >= 0)  # želimo nenegativne koordinate
        p.add_constraint(y[i] >= 0)
        p.add_constraint(x[i] + d <= p['z']) # omejimo p['z'] z največjo pokrito koordinato
        p.add_constraint(y[i] + d <= p['z'])
        for j , g in enumerate(kvadrati):
            p.add_constraint(x[i] + d <= x[j] + zg_meja * a[i,j])
            p.add_constraint(x[j] + g <= x[i] + zg_meja * a[j,i])
            p.add_constraint(y[i] + d <= y[j] + zg_meja * b[i,j])
            p.add_constraint(y[j] + g <= y[i] + zg_meja * b[j,i])
    for i, j in Combinations(range(len(kvadrati)), 2):
        p.add_constraint(a[i, j] + a[j, i] + b[i, j] + b[j, i] <= 3 + L[i,j])
    for C in Combinations(range(len(kvadrati)), (k+1)):
        p.add_constraint(sum(sum(L[C[i],C[j]] for j in range(i+1,k+1)) for i in range(k +1 )) <= comb((k+1),2)-1)
    z = p.solve()
    xx = p.get_values(x)
    yy = p.get_values(y)
    return (z, [(xx[i], yy[i], d) for i, d in enumerate(kvadrati)]) 